In [1]:
import numpy as np
import pandas as pd
from keras.models import load_model
import keras.backend as K
from sklearn.model_selection import KFold
from keras.layers import Input, Dense, Conv1D, Flatten, MaxPooling1D, Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Reshape, normalization
#from keras.models import Model
from keras.utils import to_categorical
from keras.layers.recurrent import LSTM
from sklearn import metrics
import random
from keras.models import model_from_json

Using TensorFlow backend.
F:\anaconda\anaconda\envs\tensorflow\lib\site-packages\sklearn\utils\fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
def precision(y_true, y_pred):
    # Calculates the precision
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    # Calculates the recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(test_Y, pre_test_y):
    """F1-score"""
    Precision = precision(test_Y, pre_test_y)
    Recall = recall(test_Y, pre_test_y)
    f1 = 2 * ((Precision * Recall) / (Precision + Recall + K.epsilon()))
    return f1 

def TP(test_Y,pre_test_y):
    TP = K.sum(K.round(K.clip(test_Y * pre_test_y, 0, 1)))#TP
    return TP

def FN(test_Y,pre_test_y):
    TP = K.sum(K.round(K.clip(test_Y * pre_test_y, 0, 1)))#TP
    P=K.sum(K.round(K.clip(test_Y, 0, 1)))
    FN = P-TP #FN=P-TP
    return FN

def TN(test_Y,pre_test_y):
    TN=K.sum(K.round(K.clip((test_Y-K.ones_like(test_Y))*(pre_test_y-K.ones_like(pre_test_y)), 0, 1)))#TN
    return TN

def FP(test_Y,pre_test_y):
    N = (-1)*K.sum(K.round(K.clip(test_Y-K.ones_like(test_Y), -1, 0)))#N
    TN=K.sum(K.round(K.clip((test_Y-K.ones_like(test_Y))*(pre_test_y-K.ones_like(pre_test_y)), 0, 1)))#TN
    FP=N-TN
    return FP


data = np.array(pd.read_csv("3_test_vecs.csv"))
pos_number = 10 # NOTE: the number of postive sample in test file
    
X1 = data[0:pos_number, 1:]
Y1 = data[0:pos_number, 0]
X2 = data[pos_number:, 1:]
Y2 = data[pos_number:, 0]
X = np.concatenate([X1, X2], 0)
Y = np.concatenate([Y1, Y2], 0)
#Y = Y.reshape((Y.shape[0], -1))
X = np.expand_dims(X, 2)
print (X)
print ("X.shape: ", X.shape)
print ("Y.shape: ", Y.shape)

model_name = 'CNN_model.h5'
model_back = load_model(model_name,
                        custom_objects={'precision': precision,'recall':recall,'f1':f1,'TP':TP,'FN':FN,'TN':TN,'FP':FP})
# model = load_model('pcsf.h5')
accuracy = model_back.evaluate(X,Y)
# print 'loss', loss
print ('accuracy', accuracy)
maxprobability = model_back.predict(X)
np.set_printoptions(threshold=np.inf)
print (maxprobability)
predictclass = model_back.predict(X)
predictclass = np.argmax(predictclass,axis=1)
print (predictclass)

[[[ 0.12206   ]
  [-0.162339  ]
  [-0.0600364 ]
  ..., 
  [ 0.0553366 ]
  [-0.100643  ]
  [ 0.0914919 ]]

 [[-0.00554577]
  [ 0.187162  ]
  [-0.105255  ]
  ..., 
  [-0.0919779 ]
  [-0.0929276 ]
  [ 0.052444  ]]

 [[ 0.244697  ]
  [ 0.293217  ]
  [-0.0971591 ]
  ..., 
  [-0.0711738 ]
  [-0.0810454 ]
  [ 0.0470242 ]]

 ..., 
 [[ 0.358097  ]
  [ 0.0905413 ]
  [ 0.0331105 ]
  ..., 
  [ 0.0624149 ]
  [ 0.0660773 ]
  [-0.0471668 ]]

 [[ 0.50615   ]
  [ 0.328114  ]
  [-0.0362345 ]
  ..., 
  [-0.00577319]
  [ 0.107257  ]
  [-0.10438   ]]

 [[ 0.278387  ]
  [ 0.150123  ]
  [ 0.00274092]
  ..., 
  [-0.115661  ]
  [ 0.135467  ]
  [-0.118267  ]]]
X.shape:  (20, 200, 1)
Y.shape:  (20,)
20/20 [==============================] - 0s 1ms/step
accuracy [0.19218169152736664, 0.69999998807907104, 0.6428571343421936, 0.89999997615814209, 0.74999988079071045, 9.0, 1.0, 5.0, 5.0]
[[ 0.90504688]
 [ 0.85448664]
 [ 0.81821752]
 [ 0.82772362]
 [ 0.49430528]
 [ 0.98519313]
 [ 0.79041398]
 [ 0.80507296]
 [ 0.662257